## Question 1.1.

In [ ]:
# this line is a comment. In this cell, we are bringing in Julia packages to use in this notebook.
using Cropbox, Dates  

### Q1.1b
In the next blank cell, type in the Julia code from Question 1.1 (b) and see what it does. When done, press `shift-Enter` to run the cell. 

In [ ]:
"Douglas-fir height growth model 1"
@system Q1b(Controller) begin    
    t(context.clock.time) ~ track(u"yr")
    r:  growth_rate    => 0.25 ~ preserve(parameter, u"m/yr")
    H0: initial_height => 1.5  ~ preserve(parameter, u"m")
    H(r): height       => r    ~ accumulate(u"m", init = H0)
end

In [ ]:
c = @config (
    :Clock => (;
       :step => 1u"yr",
        ),
    )
    

Let's check the definition of `Q1b` system we just created.

In [ ]:
?Q1b

Simple ploting of the model behavior

In [ ]:
visualize([Q1b], :t, :H; config = c, stop = 27u"yr", kind = :line)

### Q1.1c
First think about and jot down an analytical solution of Eqn 1.1. Then examine and run the next two cells.

In [ ]:
"Douglas-fir height growth model 2"
@system Q1c(Controller) begin
    t(context.clock.time) ~ track(u"yr")    
    yr(t): calendar_year => begin 1983u"yr" + t end ~ track(u"yr")  # This is a newly added line
    r:  growth_rate    => 0.25 ~ preserve(parameter, u"m/yr")
    H0: initial_height         => 1.5 ~ preserve(parameter, u"m")
    H(r, H0, t): height => begin # an analytical solution of Q1b
        H0 + r*t
    end ~ track(u"m") # note we track, not accumulate over time here
end

In [ ]:
visualize([Q1c], :yr, :H; config = c, stop = 27u"yr", kind = :line)

### Q1.1d
Now let's import data shown in figure 1.1 adpated from Porzig et al. (2014) and evaluate how our guessimate model performs. Data points were semi-automatically extracted using [WebPlotDigitizer](https://automeris.io/wpd/). Other tools for digitizing data from figures include [ImageJ](https://imagej.net/ij/) and [PlotDigitizer](https://plotdigitizer.com).

In [ ]:
using CSV, DataFrames, DataFramesMeta # use additional packages to read and manage data

In [ ]:
dougfir = CSV.read("fig1-data.csv", DataFrame) |> unitfy; 
# keeping the correct unit is a key in Cropbox. We will talk more about this.

Let's overlay the digitized data with our model. How did it do? Are you happy with the model? How would you improve it?

In [ ]:
p = visualize(dougfir, :year, :height; )
visualize!(p, Q1c, :yr, :H; stop=27u"yr", kind = :line)